# Competição DSA 2019/06
<h3>Objetivo: Prever Índice de Lealdade</h3>
<h4>Informações gerais disponíveis no <a href="https://www.kaggle.com/t/84f9cb851e9a428fa9fe54644df35f5c">Kaggle</a> </h4>
<h4>Parte 2: processo de ML</h4>

In [1]:
# Importar as bibliotecas necessárias para este notebook
import pandas as pd
import numpy  as np

import xgboost as xgb
from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.pipeline      import Pipeline
import math

In [2]:
# Importar os arquivos criados na parte 1
df_teste      = pd.read_csv('df_teste.csv')
df_treino     = pd.read_csv('df_treino.csv')

<h3>Ajustes finais para o processo de ML</h3>

In [3]:
df_treino.head()

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,alvo,oldest_Activation,latest_Activation
0,C_ID_92a2005557,0,0,0,0,1,0,1,0,1,-0.820283,0.071233,0.909589
1,C_ID_3d0044924f,0,0,0,1,0,1,0,0,0,0.392913,0.013699,1.241096
2,C_ID_d639edf6cd,0,1,0,0,0,0,1,0,0,0.688056,0.446575,1.739726
3,C_ID_186d6a6901,0,0,0,1,0,0,0,1,0,0.142495,0.068493,0.624658
4,C_ID_cdbd2c0db2,1,0,0,0,0,0,0,1,0,-0.159749,0.030137,0.487671


In [4]:
df_teste.head()

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,oldest_Activation,latest_Activation
0,C_ID_0ab67a22ab,0,0,1,0,0,0,0,1,1,0.008219,0.912329
1,C_ID_130fd0cbdd,0,1,0,0,0,0,0,1,0,0.032877,1.298630
2,C_ID_b709037bc5,0,0,0,0,1,1,0,0,1,0.065753,0.580822
3,C_ID_d27d835a9f,0,1,0,0,0,1,0,0,0,0.008219,0.375342
4,C_ID_2b5e3df5c2,0,0,0,0,1,1,0,0,1,1.093151,2.271233


In [5]:
df_treino['target'] = df_treino['alvo']
df_treino.drop('alvo', axis = 1, inplace = True)

In [6]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 13 columns):
card_id              201917 non-null object
feature_1_1          201917 non-null int64
feature_1_2          201917 non-null int64
feature_1_3          201917 non-null int64
feature_1_4          201917 non-null int64
feature_1_5          201917 non-null int64
feature_2_1          201917 non-null int64
feature_2_2          201917 non-null int64
feature_2_3          201917 non-null int64
feature_3_1_0        201917 non-null int64
oldest_Activation    201917 non-null float64
latest_Activation    201917 non-null float64
target               201917 non-null float64
dtypes: float64(3), int64(9), object(1)
memory usage: 20.0+ MB


In [7]:
# Preparar os dados para normalização e ML
colunas = ['feature_1_1', 'feature_1_2', 'feature_1_3', 'feature_1_4',
       'feature_1_5', 'feature_2_1', 'feature_2_2', 'feature_2_3',
       'feature_3_1_0', 'oldest_Activation', 'latest_Activation']

In [8]:
# aplicar o objeto PCA aos dataFrames
X = df_treino[colunas]
y = df_treino['target']

In [9]:
# Criar objeto DMatrix
data_dmatrix = xgb.DMatrix(data=X,label=y)

/home/angelo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/angelo/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [10]:
# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [11]:
# Criar o objeto XGB
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [12]:
# Treinar o modelo e fazer predições teste para verificar a acurácia
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

[18:43:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [14]:
# Acurácia
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 3.726287


In [19]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.05,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=100,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

[18:51:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:51:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:51:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:51:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:51:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:51:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:51:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:51:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [20]:
print((cv_results["test-rmse-mean"]).tail(1))

49    3.831991
Name: test-rmse-mean, dtype: float64


In [22]:
# Fazer previsões no modelo treinado
predictions = xg_reg.predict(df_teste[colunas])

In [23]:
df_teste['target'] = predictions

In [24]:
df_teste.head()

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,oldest_Activation,latest_Activation,target
0,C_ID_0ab67a22ab,0,0,1,0,0,0,0,1,1,0.008219,0.912329,-0.123846
1,C_ID_130fd0cbdd,0,1,0,0,0,0,0,1,0,0.032877,1.298630,-0.124002
2,C_ID_b709037bc5,0,0,0,0,1,1,0,0,1,0.065753,0.580822,-0.106343
3,C_ID_d27d835a9f,0,1,0,0,0,1,0,0,0,0.008219,0.375342,0.047462
4,C_ID_2b5e3df5c2,0,0,0,0,1,1,0,0,1,1.093151,2.271233,-0.301016


In [25]:
cols_resp = ['card_id', 'target']

In [26]:
resposta = df_teste[cols_resp]

In [27]:
resposta.head()

,card_id,target
0,C_ID_0ab67a22ab,-0.123846
1,C_ID_130fd0cbdd,-0.124002
2,C_ID_b709037bc5,-0.106343
3,C_ID_d27d835a9f,0.047462
4,C_ID_2b5e3df5c2,-0.301016


In [28]:
resposta.to_csv('respxgb.csv', index=False)